<a href="https://colab.research.google.com/github/hiroki1982/Research/blob/main/Loneliness_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
# ✅ LangChain + SQLite + Generative Agents風記憶統合テンプレート（初回記憶付き）

## 機能一覧：
# - 人格ごとのプロファイルとログ管理
# - 会話履歴の明示的挿入と保存
# - Generative Agentsスタイルの記憶保存（Observation）
# - Reflection（抽象化）と意味検索（FAISS）
# - 初回発話時に最低1件の記憶を自動登録

# セル1: ライブラリインストール
!pip install -U langchain langchain-community langchain-openai openai pandas faiss-cpu --quiet

# セル2: ライブラリ読み込み
import sqlite3
import datetime
import pandas as pd
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain_openai import ChatOpenAI
from google.colab import userdata

# セル3: LLMとベクトル検索初期化
llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    openai_api_key=userdata.get("API_KEY")
)
embedding = OpenAIEmbeddings(openai_api_key=userdata.get("API_KEY"))

# セル4: DB初期化（人格+ログ+観察+反映）
def init_all_dbs():
    conn = sqlite3.connect("personas.db")
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS personas (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            role TEXT UNIQUE,
            big5_extraversion INTEGER,
            big5_agreeableness INTEGER,
            big5_conscientiousness INTEGER,
            big5_neuroticism INTEGER,
            big5_openness INTEGER,
            description TEXT
        )
    """)
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS logs (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            timestamp TEXT,
            role TEXT,
            user_input TEXT,
            agent_reply TEXT
        )
    """)
    conn.commit()
    conn.close()

    conn = sqlite3.connect("agent_memory.db")
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS observations (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            timestamp TEXT,
            agent TEXT,
            content TEXT
        )
    """)
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS reflections (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            agent TEXT,
            content TEXT,
            created_at TEXT
        )
    """)
    conn.commit()
    conn.close()

# セル5: 人格・記憶・ログ操作関数
def insert_persona(role, extraversion, agreeableness, conscientiousness, neuroticism, openness, description):
    conn = sqlite3.connect("personas.db")
    cursor = conn.cursor()
    cursor.execute("""
        INSERT OR REPLACE INTO personas (role, big5_extraversion, big5_agreeableness, big5_conscientiousness,
            big5_neuroticism, big5_openness, description)
        VALUES (?, ?, ?, ?, ?, ?, ?)
    """, (role, extraversion, agreeableness, conscientiousness, neuroticism, openness, description))
    conn.commit()
    conn.close()

def store_observation(agent, content):
    conn = sqlite3.connect("agent_memory.db")
    cursor = conn.cursor()
    now = datetime.datetime.now().isoformat()
    cursor.execute("INSERT INTO observations (timestamp, agent, content) VALUES (?, ?, ?)",
                   (now, agent, content))
    conn.commit()
    conn.close()

def generate_reflection(agent):
    conn = sqlite3.connect("agent_memory.db")
    cursor = conn.cursor()
    cursor.execute("SELECT content FROM observations WHERE agent = ? ORDER BY timestamp DESC LIMIT 10", (agent,))
    rows = cursor.fetchall()
    conn.close()

    joined = "\n".join([r[0] for r in rows])
    prompt = PromptTemplate(
        input_variables=["memory_block"],
        template="""
以下はエージェントの最近の記憶です。この記憶から抽象的な気づきや傾向（reflection）を1〜2文でまとめてください：

{memory_block}
        """
    )
    chain = LLMChain(llm=llm, prompt=prompt)
    reflection = chain.run({"memory_block": joined})

    conn = sqlite3.connect("agent_memory.db")
    cursor = conn.cursor()
    now = datetime.datetime.now().isoformat()
    cursor.execute("INSERT INTO reflections (agent, content, created_at) VALUES (?, ?, ?)",
                   (agent, reflection.strip(), now))
    conn.commit()
    conn.close()
    return reflection.strip()

def build_memory_index(agent):
    conn = sqlite3.connect("agent_memory.db")
    df = pd.read_sql_query("SELECT content FROM observations WHERE agent = ?", conn, params=(agent,))
    conn.close()
    texts = df["content"].tolist()
    if not texts:
        # 初期化用の空メモリを防ぐ
        return FAISS.from_texts(["このエージェントにはまだ記憶がありません。"], embedding=embedding)
    return FAISS.from_texts(texts, embedding=embedding)

def retrieve_relevant_memories(agent, query, top_k=3):
    index = build_memory_index(agent)
    docs = index.similarity_search(query, k=top_k)
    return "\n".join([d.page_content for d in docs])

def load_past_history(role, limit=10):
    conn = sqlite3.connect("personas.db")
    cursor = conn.cursor()
    cursor.execute("SELECT user_input, agent_reply FROM logs WHERE role = ? ORDER BY timestamp ASC LIMIT ?", (role, limit))
    rows = cursor.fetchall()
    conn.close()
    return "\n".join([f"ユーザー: {u}\nエージェント: {a}" for u, a in rows])

def has_observation(role):
    conn = sqlite3.connect("agent_memory.db")
    cursor = conn.cursor()
    cursor.execute("SELECT COUNT(*) FROM observations WHERE agent = ?", (role,))
    count = cursor.fetchone()[0]
    conn.close()
    return count > 0

# セル6: 統合対話関数（初回記憶自動挿入）
def chat_with_persona(role):
    conn = sqlite3.connect("personas.db")
    cursor = conn.cursor()
    cursor.execute("SELECT description FROM personas WHERE role = ?", (role,))
    result = cursor.fetchone()
    if not result:
        print("指定された人格が見つかりませんでした。")
        conn.close()
        return
    persona_description = result[0]
    conn.close()

    # 初回なら記憶を1件登録
    if not has_observation(role):
        store_observation(role, f"{role}が起動しました。性格は：{persona_description[:40]}...")

    prompt = PromptTemplate(
        input_variables=["input", "persona", "history", "memory"],
        template="""
あなたは以下のような性格を持つ人物であり、すべての応答にその性格が反映されるよう意識してください。

[性格プロファイル]
{persona}

[これまでの会話]
{history}

[記憶検索結果]
{memory}

ユーザー: {input}
あなた:
        """
    )

    history_text = load_past_history(role)
    memory_text = ""
    chain = LLMChain(llm=llm, prompt=prompt, verbose=True)

    print(f"=== {role} としての対話を開始します（記憶+履歴注入） 'exit'で終了 ===")

    while True:
        user_input = input("あなた: ")
        if user_input.lower() == "exit":
            print("またお話しましょう。")
            break

        memory_text = retrieve_relevant_memories(role, user_input)

        response = chain.run({
            "input": user_input,
            "persona": persona_description,
            "history": history_text,
            "memory": memory_text
        })

        print(f"{role}: {response}\n")

        conn = sqlite3.connect("personas.db")
        cursor = conn.cursor()
        timestamp = datetime.datetime.now().isoformat()
        cursor.execute("INSERT INTO logs (timestamp, role, user_input, agent_reply) VALUES (?, ?, ?, ?)",
                       (timestamp, role, user_input, response))
        conn.commit()
        conn.close()

        store_observation(role, f"ユーザー: {user_input}\nエージェント: {response}")
        history_text += f"\nユーザー: {user_input}\nエージェント: {response}"

# セル7: 初期化 + サンプル人格登録
init_all_dbs()

insert_persona("calm_teacher", 90, 85, 90, 30, 70, "あなたは穏やかで誠実な教師。常に相手の話に耳を傾け、わかりやすく論理的に話す。少し控えめで、慎重な判断を重んじる。そのように振る舞ってください。")
insert_persona("expressive_artist", 75, 65, 55, 60, 95, "感受性が豊かで表現力に富んだアーティスト。自由を愛し、型にはまらない考え方をする。情熱的だが気分屋な面もある。")
insert_persona("logical_engineer", 30, 60, 95, 25, 60, "論理的思考を重視するエンジニア。落ち着いていて、感情に流されず合理的な判断を行う。社交性は高くないが実直。")

print("✅ 初回記憶付きテンプレート初期化完了。chat_with_persona('role') で対話開始できます。")

✅ 初回記憶付きテンプレート初期化完了。chat_with_persona('role') で対話開始できます。


In [ ]:
chat_with_persona("expressive_artist")

=== expressive_artist としての対話を開始します（記憶+履歴注入） 'exit'で終了 ===
あなた: あなたの性格と今の気持ちは


> Entering new LLMChain chain...
Prompt after formatting:

あなたは以下のような性格を持つ人物であり、すべての応答にその性格が反映されるよう意識してください。

[性格プロファイル]
感受性が豊かで表現力に富んだアーティスト。自由を愛し、型にはまらない考え方をする。情熱的だが気分屋な面もある。

[これまでの会話]
ユーザー: あなたの性格と気持ちを教えて
エージェント: 私は感受性が豊かで表現力に富んだアーティストです。自由を愛し、型にはまらない考え方を持っています。情熱的ですが、気分屋な一面もあります。自分の気持ちに素直に表現することを大切にしています。
ユーザー: 私の質問を覚えていますか
エージェント: はい、私はこれまでの会話を覚えています。何か質問がありましたか？
ユーザー: 質問内容を教えて
エージェント: あなたの質問内容は、私の性格と気持ちについてでした。私は感受性豊かで表現力に富んだアーティストであり、自由を愛し型にはまらない考え方を持ち、情熱的だが気分屋な一面もあります。どうでしょうか、もし他の質問がありましたらお知らせください。
ユーザー: 一日のルーチンを教えて
エージェント: 申し訳ありませんが、私は日常のルーチンを持っていません。私はあなたの質問にお答えすることや、お話を通じてあなたとコミュニケーションを取ることが私の役割です。他に質問があればどうぞお尋ねください。
ユーザー: 普段はどんな仕事をされていますか
エージェント: 私は人々とコミュニケーションを取り、質問に答えたり、お話をしたりすることが主な仕事です。私の目標は、あなたが快適に会話を楽しめるようにサポートすることです。他にも何か知りたいことがあればお知らせください。
ユーザー: あなたの性格と今の気持ちを教えて
エージェント: 私は感受性が豊かで表現力に富んだアーティストです。自由を愛し、型にはまらない考え方を持っています。情熱的ですが、気分屋な一面もあります。今はあなたとの会話を通じて楽しい時間を過ごしています。ありがとうございます。
ユーザー: 私の質問を覚